# 맛집 추천  시스템 

## 데이터 전처리 작업

In [1]:
import pandas as pd

#### 데이터 불러오기

In [2]:
data = pd.read_csv('./data/네이버맵음식점평점_최종.csv')

In [3]:
data.head()

,Unnamed: 0,name,upjong,address,category,rating,write_id
0,0,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5dee59b58f87a842bc875a3a
1,4,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5c381b1c5745ec219264a906
2,8,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5df37ace8f87a842bc889e8c
3,12,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5d4fac00a2b372d9cf8c57be
4,16,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5f05c9d4699d39675d9e8551


In [4]:
data.drop(columns={'Unnamed: 0', 'write_id'}, inplace=True)

#### 리뷰 갯수

In [5]:
cnt_review = pd.DataFrame(data.value_counts('name'))

In [6]:
cnt_review.head()

,0
name,
고집돌우럭 중문점,5845
문개항아리조천본점,5188
설레임,5042
델문도,3183
제주순풍해장국함덕점,2961


#### 평균 별점

In [7]:
avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

In [8]:
avg_star.head()

,rating
name,
(주)삼다도횟집,4.350000
09학번,5.000000
1.5커피,5.000000
11시11분,4.684211
168테우카페,4.650000


## 맛집 추천 시스템 함수화(가중 평점 기반)

#### 전처리 및 가중치 함수화

In [9]:
def make_review_data(data, area, percentile=0.6):

    cnt_review = pd.DataFrame(data.value_counts('name'))
    avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

    # 평점/리뷰갯수 정보를 가지는 temp 변수 생성
    temp = avg_star.merge(cnt_review,left_index=True,right_index=True,how='left')
    temp.columns = ['rating','num']
    
    temp= temp.fillna(0)
    
    temp

    
    m = temp['num'].quantile(percentile)
    C = temp['rating'].mean()

    
    #가충치 함수를 람다함수로 처리
    temp['weighted_rating'] = temp.apply((lambda x: (x['num']/(x['num']+m)*x['rating'])+(m/(m+x['num'])*C)), axis=1)
    
    
    ## 주소 부분
    # 음식점 이름 + 주소를 기준으로 그룹화 시켜서 
    address =pd.DataFrame(data.groupby(['name'])['address'].first())
    
    # 주소 부분 병합
    temp=temp.merge(address,left_index=True,right_index=True)
    
    
    ## 지역부분
    # 음식점 이름 + 지역을 기준으로 그룹화 시켜서 
    name_area =pd.DataFrame(data.groupby(['name'])['category'].first())
    
    # 평점/리뷰갯수에 음식점 이름 + 지역 병합
    temp=temp.merge(name_area,left_index=True,right_index=True)
    
    
    # 사용자 위치에 해당되는 정보만 정리
    temp=temp[temp['category']==area]
    
    temp
    # 정렬후 반환
    return temp[['rating','weighted_rating','num','category','address']].sort_values('weighted_rating', ascending=False)[:10]

## 실제 구현 부분

In [49]:
# 해당 지점을 DB랑 연결
data = pd.read_csv('./data/네이버맵음식점평점_최종.csv')

if __name__=='__main__':
    print('사용자 위치를 입력해주세요')
    print()
    input_data_area=input()
    # 식당 순위 info에 저장
    info = make_review_data(data, input_data_area)
    
    info.reset_index(inplace=True)
    
    print()
    print()
    # 식당 순위 출력
    for i in range(10):
        print(i+1,'순위')
        print('음식점 이름 : ', info['name'][i], '\t평균 평점 : ', round(info['weighted_rating'][i],2))
        print()
        print()

사용자 위치를 입력해주세요

애월읍


1 순위
음식점 이름 :  쉬다가게 	평균 평점 :  4.92


2 순위
음식점 이름 :  호커센터 	평균 평점 :  4.91


3 순위
음식점 이름 :  제주기와 	평균 평점 :  4.9


4 순위
음식점 이름 :  포구횟집 	평균 평점 :  4.88


5 순위
음식점 이름 :  누렁소앤도새기 	평균 평점 :  4.88


6 순위
음식점 이름 :  카페앤틱애월 	평균 평점 :  4.86


7 순위
음식점 이름 :  낮갈치밤돼지 	평균 평점 :  4.84


8 순위
음식점 이름 :  와썹 	평균 평점 :  4.84


9 순위
음식점 이름 :  금돈가 	평균 평점 :  4.83


10 순위
음식점 이름 :  애월포구횟집 	평균 평점 :  4.82




In [11]:
info

,name,rating,weighted_rating,num,category,address
0,쉬다가게,4.960648,4.923976,216,애월읍,"제주특별자치도 제주시 애월읍 구엄길 66, 1층"
1,호커센터,4.920394,4.906717,559,애월읍,제주특별자치도 제주시 애월읍 애월로11길 25-2
2,제주기와,4.913000,4.897989,500,애월읍,"제주특별자치도 제주시 애월읍 광령남4길 45-1, 1층"
3,포구횟집,4.898798,4.884199,499,애월읍,제주특별자치도 제주시 애월읍 가문동길 43
4,누렁소앤도새기,4.948980,4.875987,98,애월읍,"제주특별자치도 제주시 애월읍 광령남6길 14, 1층"
5,카페앤틱애월,4.975806,4.863622,62,애월읍,"제주특별자치도 제주시 애월읍 하귀11길 39, 1층"
6,낮갈치밤돼지,4.914773,4.840024,88,애월읍,"제주특별자치도 제주시 애월읍 신엄7길 9, 1층"
7,와썹,4.933824,4.837649,68,애월읍,"제주특별자치도 제주시 애월읍 곽지1길 30, 1층 제1호"
8,금돈가,4.857488,4.826738,207,애월읍,"제주특별자치도 제주시 애월읍 가문동남길 63, 1층"
9,애월포구횟집,4.875000,4.815521,104,애월읍,"제주특별자치도 제주시 애월읍 애월로11길 22, 2층"


In [12]:
 make_review_data(data, input_data_area)

,rating,weighted_rating,num,category,address
name,,,,,
쉬다가게,4.960648,4.923976,216,애월읍,"제주특별자치도 제주시 애월읍 구엄길 66, 1층"
호커센터,4.920394,4.906717,559,애월읍,제주특별자치도 제주시 애월읍 애월로11길 25-2
제주기와,4.913000,4.897989,500,애월읍,"제주특별자치도 제주시 애월읍 광령남4길 45-1, 1층"
포구횟집,4.898798,4.884199,499,애월읍,제주특별자치도 제주시 애월읍 가문동길 43
누렁소앤도새기,4.948980,4.875987,98,애월읍,"제주특별자치도 제주시 애월읍 광령남6길 14, 1층"
카페앤틱애월,4.975806,4.863622,62,애월읍,"제주특별자치도 제주시 애월읍 하귀11길 39, 1층"
낮갈치밤돼지,4.914773,4.840024,88,애월읍,"제주특별자치도 제주시 애월읍 신엄7길 9, 1층"
와썹,4.933824,4.837649,68,애월읍,"제주특별자치도 제주시 애월읍 곽지1길 30, 1층 제1호"
금돈가,4.857488,4.826738,207,애월읍,"제주특별자치도 제주시 애월읍 가문동남길 63, 1층"


In [13]:
 make_review_data(data, '서귀포시')

,rating,weighted_rating,num,category,address
name,,,,,
성산흑돈쭐,4.972973,4.941318,259,서귀포시,제주특별자치도 서귀포시 성산읍 성산중앙로 72
대한민육,4.959834,4.937301,361,서귀포시,제주특별자치도 서귀포시 서문로 30 (서귀동)
흑돼지해물삼합,4.927785,4.922947,1634,서귀포시,제주특별자치도 서귀포시 태평로482번길 50 (서귀동)
중문색달통갈치,4.902311,4.894486,952,서귀포시,"제주특별자치도 서귀포시 일주서로 993, 1층 101호 (색달동)"
카페블루하우스,4.895620,4.888763,1073,서귀포시,"제주특별자치도 서귀포시 중앙로 73, 1층 (서귀동)"
중문흑돼지천국,4.890385,4.881109,780,서귀포시,"제주특별자치도 서귀포시 이어도로 137, 1층 (대포동)"
만족한상회,4.890299,4.879537,670,서귀포시,"제주특별자치도 서귀포시 중문상로 58-5, 1층 (중문동)"
칼초네화덕피자,4.887967,4.878028,723,서귀포시,제주특별자치도 서귀포시 예래로 115 (상예동)
포트그릴스테이크하우스,4.898268,4.867864,231,서귀포시,"제주특별자치도 서귀포시 솔동산로10번길 48, 1층 (서귀동)"


## 맛집 추천 시스템(컨텐츠 기반)

In [ ]:
def make_relation(data):
    
    # 사용자 별 식당 별점 피봇테이블 -  개오래걸림
    piv = data.pivot_table(values='rating',index='write_id',columns='name')
    
    # NaN 값을 모두 0 으로 변환 - 오래걸림
    piv = piv.fillna(0)
    
    # 사용자-아이템 행렬 데이터를 아이템-사용자 행렬 데이터로 변환
    piv_T = piv.transpose()
    
    # 아이템간의 코사인 유사도로 아이템 유사도 산출 - 좀 걸림
    from sklearn.metrics.pairwise import cosine_similarity
    piv_sim = cosine_similarity(piv_T,piv_T)

    # cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
    piv_sim_df = pd.DataFrame(data=piv_sim, index=piv.columns,
                              columns=piv.columns)
    
    return piv_sim_df

In [ ]:
piv_sim_df=make_relation(data)

In [25]:
# 매번 돌리면 오래걸리기 때문에 저장 후 사용

piv_sim_df.to_csv('data/relation_food.csv')

In [30]:
data2 = pd.read_csv('data/relation_food.csv',index_col=['name'])

## 실제 구현 부분

In [ ]:
# name과 유사도가 높은 상위 10개 식당 추출

data2 = pd.read_csv('data/relation_food.csv',index_col=['name'])

In [72]:
# 해당 지점을 DB랑 연결

if __name__=='__main__':
    print('만족한 식당을 입력해 주세요')
    print()
    input_data_rest=input()
    # 관련 식당 info에 저장
    info = data2[input_data_rest].sort_values(ascending=False)[1:11]
    info = pd.DataFrame(info).merge(pd.DataFrame(data.groupby(['name'])['address'].first()),left_index=True,right_on='name',how='inner')
    print()
    print()
    
    # 관련 식당 순위 출력
    for i in range(10):
        print(i+1,'순위')
        print('음식점 이름 : ', info.index[i], '\t연관성 : ', round(info[input_data_rest][i],2))
        print()
        print()

만족한 식당을 입력해 주세요

해미원횟집


1 순위
음식점 이름 :  기쁨이네해물탕앤돌솥밥 	연관성 :  0.37


2 순위
음식점 이름 :  우돈향 	연관성 :  0.37


3 순위
음식점 이름 :  신우성흑돼지 	연관성 :  0.35


4 순위
음식점 이름 :  중문회포장센터새벽야시장 	연관성 :  0.34


5 순위
음식점 이름 :  성산흑돈쭐 	연관성 :  0.33


6 순위
음식점 이름 :  연탄과친한돼지 	연관성 :  0.32


7 순위
음식점 이름 :  제주정지 	연관성 :  0.31


8 순위
음식점 이름 :  대한민육 	연관성 :  0.29


9 순위
음식점 이름 :  돈꼬집 	연관성 :  0.28


10 순위
음식점 이름 :  만세본점 	연관성 :  0.28




In [32]:
# name과 유사도가 높은 상위 6개 식당 추출
data2['성산흑돈쭐'].sort_values(ascending=False)[1:7]

name
우돈향            0.437549
기쁨이네해물탕앤돌솥밥    0.379450
애월포구횟집         0.331223
해미원횟집          0.325875
연탄과친한돼지        0.325105
만세본점           0.307287
Name: 성산흑돈쭐, dtype: float64

In [35]:
aa=data2['성산흑돈쭐'].sort_values(ascending=False)[1:7]

In [69]:
pd.DataFrame(info).merge(pd.DataFrame(data.groupby(['name'])['address'].first()),left_index=True,right_on='name',how='inner')

,성산흑돈쭐,address
name,,
우돈향,0.437549,"제주특별자치도 제주시 조천읍 조함해안로 530, 2층"
기쁨이네해물탕앤돌솥밥,0.379450,제주특별자치도 제주시 조천읍 신북로 527-1
애월포구횟집,0.331223,"제주특별자치도 제주시 애월읍 애월로11길 22, 2층"
해미원횟집,0.325875,제주특별자치도 서귀포시 표선면 민속해안로 578-2
연탄과친한돼지,0.325105,"제주특별자치도 제주시 원노형로 61, 1층 (노형동)"
만세본점,0.307287,제주특별자치도 제주시 오라로 152 (오라일동)
신우성흑돼지,0.284904,"제주특별자치도 서귀포시 중문관광로72번길 29-9, 1층 (색달동)"
제주정지,0.283028,"제주특별자치도 제주시 서해안로 480, 1층 (용담삼동)"
꿀맛,0.280240,"제주특별자치도 서귀포시 막숙포로 95, 1층 (법환동)"


In [73]:
info

,해미원횟집,address
name,,
기쁨이네해물탕앤돌솥밥,0.373190,제주특별자치도 제주시 조천읍 신북로 527-1
우돈향,0.373052,"제주특별자치도 제주시 조천읍 조함해안로 530, 2층"
신우성흑돼지,0.353222,"제주특별자치도 서귀포시 중문관광로72번길 29-9, 1층 (색달동)"
중문회포장센터새벽야시장,0.338652,제주특별자치도 서귀포시 중문관광로 293 (대포동)
성산흑돈쭐,0.325875,제주특별자치도 서귀포시 성산읍 성산중앙로 72
연탄과친한돼지,0.317279,"제주특별자치도 제주시 원노형로 61, 1층 (노형동)"
제주정지,0.307003,"제주특별자치도 제주시 서해안로 480, 1층 (용담삼동)"
대한민육,0.292306,제주특별자치도 서귀포시 서문로 30 (서귀동)
돈꼬집,0.277285,제주특별자치도 서귀포시 신서로32번길 18 (강정동)
